GT4Py - GridTools for Python

Copyright (c) 2014-2023, ETH Zurich
All rights reserved.

This file is part the GT4Py project and the GridTools framework.
GT4Py is free software: you can redistribute it and/or modify it under
the terms of the GNU General Public License as published by the
Free Software Foundation, either version 3 of the License, or any later
version. See the LICENSE.txt file at the top-level directory of this
distribution for a copy of the license or check <https://www.gnu.org/licenses/>.

SPDX-License-Identifier: GPL-3.0-or-later

# Comparison gt4py.cartesian vs. gt4py.next

Imports

In [1]:
import numpy as np

nx = 32
ny = 32
nz = 1
dtype = np.float64

Storages
--

In [2]:
import gt4py.next as gtx

allocator = gtx.itir_embedded # should match the executor

I = gtx.Dimension("I")
J = gtx.Dimension("J")
K = gtx.Dimension("K")

domain = gtx.domain({I: nx, J: ny, K: nz})

inp = gtx.as_field(domain, np.fromfunction(lambda x, y, z: x**2+y**2, shape=(nx, ny, nz)), dtype, allocator=allocator)
out_cartesian = gtx.zeros(domain, dtype, allocator=allocator)
out_next = gtx.zeros(domain, dtype, allocator=allocator)

gt4py.cartesian
--

In [3]:
import gt4py.cartesian.gtscript as gtscript

cartesian_backend = "numpy"
# cartesian_backend = "gt:cpu_ifirst"

@gtscript.stencil(backend=cartesian_backend)
def lap_cartesian(
    inp: gtscript.Field[dtype],
    out: gtscript.Field[dtype],
):
    with computation(PARALLEL), interval(...):
        out = -4.0 * inp[0, 0, 0] + inp[-1, 0, 0] + inp[1, 0, 0] + inp[0, -1, 0] + inp[0, 1, 0]

lap_cartesian(inp=inp, out=out_cartesian, origin=(1, 1, 0), domain=(nx-2, ny-2, nz))

In [4]:
from gt4py.next import Field

backend = gtx.itir_embedded
# backend = cpu_backend

Ioff = gtx.FieldOffset("I", source=I, target=(I,))
Joff = gtx.FieldOffset("J", source=J, target=(J,))

@gtx.field_operator
def lap_next(inp: Field[[I, J, K], dtype]) -> Field[[I, J, K], dtype]:
    return -4.0 * inp + inp(Ioff[-1]) + inp(Ioff[1]) + inp(Joff[-1]) + inp(Joff[1])

@gtx.program(backend=backend)
def lap_next_program(inp: Field[[I, J, K], dtype], out: Field[[I, J, K], dtype]):
    lap_next(inp, out=out[1:-1, 1:-1, :])

lap_next_program(inp, out_next, offset_provider={"Ioff": I, "Joff": J})

In [5]:
assert np.allclose(out_cartesian.asnumpy(), out_next.asnumpy())

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''